In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import pandas as pd
from pandas import json_normalize
from random import randint
from time import sleep

### Authentification

In [3]:
secrets_file = open("secrets.txt","r")

In [4]:
string = secrets_file.read()

In [5]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

#### authentication with secrets

In [6]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

### Playlist 1: https://open.spotify.com/playlist/6yPiKpy7evrwvZodByKvM9 

In [7]:
playlist1 = sp.user_playlist_tracks("spotify", "6yPiKpy7evrwvZodByKvM9")

In [8]:
# Let's look at items and total:
playlist1.keys() 

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [9]:
# let's see how many songs are in this playlist:
playlist1["total"]

10000

In [10]:
# Each page is limited to 100 tracks, we will have to fix it:
len(playlist1["items"])

100

In [11]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # respectful nap
    return tracks

In [12]:
all_tracks1 = get_playlist_tracks("6yPiKpy7evrwvZodByKvM9")
len(all_tracks1)

10000

In [13]:
pd.set_option("display.max_columns", 0)

In [14]:
tracks1 = json_normalize(all_tracks1)

In [15]:
tracks1.head()

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls.spotify,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
0,2017-02-27T01:38:09Z,False,None,https://open.spotify.com/user/12160726861,https://api.spotify.com/v1/users/12160726861,12160726861,user,spotify:user:12160726861,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/414VLYRvebT1LdG...,https://api.spotify.com/v1/albums/414VLYRvebT1...,414VLYRvebT1LdGdI9XqsQ,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","Get Physical Music Presents: Body Language, Vo...",2016-04-01,day,17.0,album,spotify:album:414VLYRvebT1LdGdI9XqsQ,[{'external_urls': {'spotify': 'https://open.s...,[],1,152560,False,False,USD8D1528015,https://open.spotify.com/track/33xMbeHzmWd6Od0...,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,33xMbeHzmWd6Od0BmLZEUs,False,2K,0,None,True,17,track,spotify:track:33xMbeHzmWd6Od0BmLZEUs,None
1,2017-02-27T01:38:09Z,False,None,https://open.spotify.com/user/12160726861,https://api.spotify.com/v1/users/12160726861,12160726861,user,spotify:user:12160726861,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/26Fac6kNgB6MLeo...,https://api.spotify.com/v1/albums/26Fac6kNgB6M...,26Fac6kNgB6MLeoGVkmpKN,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Full Circle,1972-07-17,day,9.0,album,spotify:album:26Fac6kNgB6MLeoGVkmpKN,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,197706,False,False,USEE10609618,https://open.spotify.com/track/3UnyplmZaq547hw...,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,3UnyplmZaq547hwsfOR5yy,False,4 Billion Souls,25,https://p.scdn.co/mp3-preview/d6645e0eeb0f6849...,True,2,track,spotify:track:3UnyplmZaq547hwsfOR5yy,None
2,2017-02-27T01:38:09Z,False,None,https://open.spotify.com/user/12160726861,https://api.spotify.com/v1/users/12160726861,12160726861,user,spotify:user:12160726861,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/36lJLPoPPOKNFdd...,https://api.spotify.com/v1/albums/36lJLPoPPOKN...,36lJLPoPPOKNFddTAcirnc,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",In Rainbows (Disk 2),2007-12-03,day,8.0,album,spotify:album:36lJLPoPPOKNFddTAcirnc,[{'external_urls': {'spotify': 'https://open.s...,[],1,244284,False,False,GBSTK0700018,https://open.spotify.com/track/1w8QCSDH4QobcQe...,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,1w8QCSDH4QobcQeT4uMKLm,False,4 Minute Warning,0,None,True,8,track,spotify:track:1w8QCSDH4QobcQeT4uMKLm,None
3,2017-02-27T01:38:09Z,False,None,https://open.spotify.com/user/12160726861,https://api.spotify.com/v1/users/12160726861,12160726861,user,spotify:user:12160726861,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/0aPnQJ5p5nLwSHI...,https://api.spotify.com/v1/albums/0aPnQJ5p5nLw...,0aPnQJ5p5nLwSHIQH6d5YR,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Philosophy of Miracle,2001,year,12.0,album,spotify:album:0aPnQJ5p5nLwSHIQH6d5YR,[{'external_urls': {'spotify': 'https://open.s...,[],1,249939,False,False,USWMA9010661,https://open.spotify.com/track/7J9mBHG4J2eIfDA...,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,7J9mBHG4J2eIfDAv5BehKA,False,7 Element,0,None,True,2,track,spotify:track:7J9mBHG4J2eIfDAv5BehKA,None
4,2017-02-27T01:38:09Z,False,None,https

In [16]:
# we can even create a new column containing these DataFrames! KEEP song URI
def expand_list_dict(row):
    df = json_normalize(row['track.artists'])
    df['song_id'] = row['track.id']
    return df

tracks1['artists_dfs'] = tracks1.apply(expand_list_dict, axis=1)
tracks1['artists_dfs'][3]

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/0UK6JkgUMa2...,0UK6JkgUMa28b4t8eCtg6P,Vitas,artist,spotify:artist:0UK6JkgUMa28b4t8eCtg6P,https://open.spotify.com/artist/0UK6JkgUMa28b4...,7J9mBHG4J2eIfDAv5BehKA


In [17]:
# now we create a new dataframe with all these artists
artist_df1 = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri'])
for mini_df in tracks1['artists_dfs']:
    #display(val)
    artist_df1 = pd.concat([artist_df1, mini_df], axis=0)
    
artist_df1

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/0IVapwlnM3d...,0IVapwlnM3dEOiMsHXsghT,Nosaj Thing,artist,spotify:artist:0IVapwlnM3dEOiMsHXsghT,https://open.spotify.com/artist/0IVapwlnM3dEOi...,33xMbeHzmWd6Od0BmLZEUs
0,https://api.spotify.com/v1/artists/22WZ7M8sxp5...,22WZ7M8sxp5THdruNY3gXt,The Doors,artist,spotify:artist:22WZ7M8sxp5THdruNY3gXt,https://open.spotify.com/artist/22WZ7M8sxp5THd...,3UnyplmZaq547hwsfOR5yy
0,https://api.spotify.com/v1/artists/4Z8W4fKeB5Y...,4Z8W4fKeB5YxbusRsdQVPb,Radiohead,artist,spotify:artist:4Z8W4fKeB5YxbusRsdQVPb,https://open.spotify.com/artist/4Z8W4fKeB5Yxbu...,1w8QCSDH4QobcQeT4uMKLm
0,https://api.spotify.com/v1/artists/0UK6JkgUMa2...,0UK6JkgUMa28b4t8eCtg6P,Vitas,artist,spotify:artist:0UK6JkgUMa28b4t8eCtg6P,https://open.spotify.com/artist/0UK6JkgUMa28b4...,7J9mBHG4J2eIfDAv5BehKA
0,https://api.spotify.com/v1/artists/4KWTAlx2Rvb...,4KWTAlx2RvbpseOGMEmROg,R.E.M.,artist,spotify:artist:4KWTAlx2RvbpseOGMEmROg,https://open.spotify.com/artist/4KWTAlx2Rvbpse...,1VZedwJj1gyi88WFRhfThb
...,...,...,...,...,...,...,...
0,https://api.spotify.com/v1/artists/3RGLhK1IP9j...,3RGLhK1IP9jnYFH4BRFJBS,The Clash,artist,spotify:artist:3RGLhK1IP9jnYFH4BRFJBS,https://open.spotify.com/artist/3RGLhK1IP9jnYF...,5jzma6gCzYtKB1DbEwFZKH
0,https://api.spotify.com/v1/artists/3ICyfoySNDZ...,3ICyfoySNDZqtBVmaBT84I,War,artist,spotify:artist:3ICyfoySNDZqtBVmaBT84I,https://open.spotify.com/artist/3ICyfoySNDZqtB...,2fmMPJb5EzZCx8BcNJvVk4
0,https://api.spotify.com/v1/artists/3OsRAKCvk37...,3OsRAKCvk37zwYcnzRf5XF,Moby,artist,spotify:artist:3OsRAKCvk37zwYcnzRf5XF,https://open.spotify.com/artist/3OsRAKCvk37zwY...,60rIdEPDrzyLiLC0icp3xz
0,https://api.spotify.com/v1/artists/023YMawCG3O...,023YMawCG3OvACmRjWxLWC,The Cat Empire,artist,spotify:artist:023YMawCG3OvACmRjWxLWC,https://open.spotify.com/artist/023YMawCG3OvAC...,0sEm1ld0V8YTCPcjPVfIsc


In [18]:
# now we merge (join) the two dataframes, keeping only the fields we need
df_merged1 = pd.merge(left=tracks1,
                    right=artist_df1,
                    how='inner',
                    left_on='track.id',
                    right_on='song_id')
df_merged1.head()

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls.spotify,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url,artists_dfs,href,id,name,type,uri,external_urls.spotify,song_id
0,2017-02-27T01:38:09Z,False,None,https://open.spotify.com/user/12160726861,https://api.spotify.com/v1/users/12160726861,12160726861,user,spotify:user:12160726861,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/414VLYRvebT1LdG...,https://api.spotify.com/v1/albums/414VLYRvebT1...,414VLYRvebT1LdGdI9XqsQ,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","Get Physical Music Presents: Body Language, Vo...",2016-04-01,day,17.0,album,spotify:album:414VLYRvebT1LdGdI9XqsQ,[{'external_urls': {'spotify': 'https://open.s...,[],1,152560,False,False,USD8D1528015,https://open.spotify.com/track/33xMbeHzmWd6Od0...,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,33xMbeHzmWd6Od0BmLZEUs,False,2K,0,None,True,17,track,spotify:track:33xMbeHzmWd6Od0BmLZEUs,None,...,https://api.spotify.com/v1/artists/0IVapwlnM3d...,0IVapwlnM3dEOiMsHXsghT,Nosaj Thing,artist,spotify:artist:0IVapwlnM3dEOiMsHXsghT,https://open.spotify.com/artist/0IVapwlnM3dEOi...,33xMbeHzmWd6Od0BmLZEUs
1,2017-02-27T01:38:09Z,False,None,https://open.spotify.com/user/12160726861,https://api.spotify.com/v1/users/12160726861,12160726861,user,spotify:user:12160726861,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/26Fac6kNgB6MLeo...,https://api.spotify.com/v1/albums/26Fac6kNgB6M...,26Fac6kNgB6MLeoGVkmpKN,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Full Circle,1972-07-17,day,9.0,album,spotify:album:26Fac6kNgB6MLeoGVkmpKN,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,197706,False,False,USEE10609618,https://open.spotify.com/track/3UnyplmZaq547hw...,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,3UnyplmZaq547hwsfOR5yy,False,4 Billion Souls,25,https://p.scdn.co/mp3-preview/d6645e0eeb0f6849...,True,2,track,spotify:track:3UnyplmZaq547hwsfOR5yy,None,...,https://api.spotify.com/v1/artists/22WZ7M8sxp5...,22WZ7M8sxp5THdruNY3gXt,The Doors,artist,spotify:artist:22WZ7M8sxp5THdruNY3gXt,https://open.spotify.com/artist/22WZ7M8sxp5THd...,3UnyplmZaq547hwsfOR5yy
2,2017-02-27T01:38:09Z,False,None,https://open.spotify.com/user/12160726861,https://api.spotify.com/v1/users/12160726861,12160726861,user,spotify:user:12160726861,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/36lJLPoPPOKNFdd...,https://api.spotify.com/v1/albums/36lJLPoPPOKN...,36lJLPoPPOKNFddTAcirnc,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",In Rainbows (Disk 2),2007-12-03,day,8.0,album,spotify:album:36lJLPoPPOKNFddTAcirnc,[{'external_urls': {'spotify': 'https://open.s...,[],1,244284,False,False,GBSTK0700018,https://open.spotify.com/track/1w8QCSDH4QobcQe...,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,1w8QCSDH4QobcQeT4uMKLm,False,4 Minute Warning,0,None,True,8,track,spotify:track:1w8QCSDH4QobcQeT4uMKLm,None,...,https://api.spotify.com/v1/artists/4Z8W4fKeB5Y...,4Z8W4fKeB5YxbusRsdQVPb,Radiohead,artist,spotify:artist:4Z8W4fKeB5YxbusRsdQVPb,https://open.spotify.com/artist/4Z8W4fKeB5Yxbu...,1w8QCSDH4QobcQeT4uMKLm
3,2017-02-27T01:38:09Z,False,None,https://open.spotify.com/user/12160726861,https://api.spotify.com/v1/users/12160726861,12

In [19]:
df_final1 = df_merged1[['track.name', 'name', 'song_id']]
df_final1

,track.name,name,song_id
0,2K,Nosaj Thing,33xMbeHzmWd6Od0BmLZEUs
1,4 Billion Souls,The Doors,3UnyplmZaq547hwsfOR5yy
2,4 Minute Warning,Radiohead,1w8QCSDH4QobcQeT4uMKLm
3,7 Element,Vitas,7J9mBHG4J2eIfDAv5BehKA
4,#9 Dream,R.E.M.,1VZedwJj1gyi88WFRhfThb
...,...,...,...
18591,London Calling - Remastered,The Clash,5jzma6gCzYtKB1DbEwFZKH
18592,Low Rider,War,2fmMPJb5EzZCx8BcNJvVk4
18593,Flower,Moby,60rIdEPDrzyLiLC0icp3xz
18594,Brighter Than Gold,The Cat Empire,0sEm1ld0V8YTCPcjPVfIsc


### Audio features

In [20]:
type_counts = df_final1['song_id'].apply(type).value_counts()
print(type_counts)


<class 'str'>         11872
<class 'NoneType'>     6724
Name: song_id, dtype: int64


In [21]:
chunks = [(i, i+100) for i in range(0, len(df_final1), 100)]
chunks
audio_features_list = []
for chunk in chunks:
    df_final1['song_id'] = df_final1['song_id'].astype(str)
    id_list100 = df_final1['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,3000)/1000)
len(audio_features_list)

C:\Users\laiag\AppData\Local\Temp\ipykernel_20728\3107556052.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final1['song_id'] = df_final1['song_id'].astype(str)


18596

In [22]:
audio_features_df = json_normalize(audio_features_list)

In [23]:
audio_features_df.drop_duplicates(inplace=True)

In [24]:
df_w_audio_ft1 = pd.merge(left=df_final1,
                        right=audio_features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft1

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,2K,Nosaj Thing,33xMbeHzmWd6Od0BmLZEUs,0.310,0.445,7.0,-13.355,0.0,0.0863,0.09400,0.067800,0.1130,0.122,95.360,audio_features,33xMbeHzmWd6Od0BmLZEUs,spotify:track:33xMbeHzmWd6Od0BmLZEUs,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,https://api.spotify.com/v1/audio-analysis/33xM...,152560.0,3.0
1,4 Billion Souls,The Doors,3UnyplmZaq547hwsfOR5yy,0.419,0.565,5.0,-11.565,1.0,0.0347,0.13700,0.337000,0.1280,0.648,151.277,audio_features,3UnyplmZaq547hwsfOR5yy,spotify:track:3UnyplmZaq547hwsfOR5yy,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,https://api.spotify.com/v1/audio-analysis/3Uny...,197707.0,4.0
2,4 Minute Warning,Radiohead,1w8QCSDH4QobcQeT4uMKLm,0.354,0.302,9.0,-13.078,1.0,0.0326,0.59000,0.070900,0.1110,0.223,123.753,audio_features,1w8QCSDH4QobcQeT4uMKLm,spotify:track:1w8QCSDH4QobcQeT4uMKLm,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,https://api.spotify.com/v1/audio-analysis/1w8Q...,244285.0,4.0
3,7 Element,Vitas,7J9mBHG4J2eIfDAv5BehKA,0.727,0.785,5.0,-6.707,0.0,0.0603,0.32500,0.126000,0.3100,0.960,129.649,audio_features,7J9mBHG4J2eIfDAv5BehKA,spotify:track:7J9mBHG4J2eIfDAv5BehKA,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,https://api.spotify.com/v1/audio-analysis/7J9m...,249940.0,4.0
4,#9 Dream,R.E.M.,1VZedwJj1gyi88WFRhfThb,0.571,0.724,0.0,-5.967,1.0,0.0260,0.02310,0.003110,0.0919,0.385,116.755,audio_features,1VZedwJj1gyi88WFRhfThb,spotify:track:1VZedwJj1gyi88WFRhfThb,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,https://api.spotify.com/v1/audio-analysis/1VZe...,278320.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11867,London Calling - Remastered,The Clash,5jzma6gCzYtKB1DbEwFZKH,0.651,0.801,0.0,-7.340,1.0,0.0514,0.12300,0.000000,0.0825,0.776,133.769,audio_features,5jzma6gCzYtKB1DbEwFZKH,spotify:track:5jzma6gCzYtKB1DbEwFZKH,https://api.spotify.com/v1/tracks/5jzma6gCzYtK...,https://api.spotify.com/v1/audio-analysis/5jzm...,200480.0,4.0
11868,Low Rider,War,2fmMPJb5EzZCx8BcNJvVk4,0.811,0.647,0.0,-10.989,1.0,0.0498,0.08230,0.681000,0.0572,0.990,139.787,audio_features,2fmMPJb5EzZCx8BcNJvVk4,spotify:track:2fmMPJb5EzZCx8BcNJvVk4,https://api.spotify.com/v1/tracks/2fmMPJb5EzZC...,https://api.spotify.com/v1/audio-analysis/2fmM...,191560.0,4.0
11869,Flower,Moby,60rIdEPDrzyLiLC0icp3xz,0.686,0.610,7.0,-5.902,1.0,0.0262,0.19000,0.000931,0.0710,0.766,80.567,audio_features,60rIdEPDrzyLiLC0icp3xz,spotify:track:60rIdEPDrzyLiLC0icp3xz,https://api.spotify.com/v1/tracks/60rIdEPDrzyL...,https://api.spotify.com/v1/audio-analysis/60rI...,206293.0,4.0
11870,Brighter Than Gold,The Cat Empire,0sEm1ld0V8YTCPcjPVfIsc,0.711,0.718,6.0,-5.739,1.0,0.0380,0.00824,0.002080,0.0732,0.688,117.071,audio_features,0sEm1ld0V8YTCPcjPVfIsc,spotify:track:0sEm1ld0V8YTCPcjPVfIsc,https://api.spotify.com/v1/tracks/0sEm1ld0V8YT...,https://api.spotify.com/v1/audio-analysis/0sEm...,200293.0,4.0


### Playlist 2: https://open.spotify.com/playlist/5xqpyfZyS1DVysoevdVyEn 

In [25]:
playlist2 = sp.user_playlist_tracks("spotify", "5xqpyfZyS1DVysoevdVyEn")

In [26]:
# Let's look at items and total:
playlist2.keys() 

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [27]:
# let's see how many songs are in this playlist:
playlist2["total"]

10991

In [28]:
# Each page is limited to 100 tracks, we will have to fix it:
len(playlist2["items"])

100

In [29]:
def get_playlist_tracks(playlist_id): 
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # respectful nap
    return tracks

In [30]:
all_tracks2 = get_playlist_tracks("5xqpyfZyS1DVysoevdVyEn")
len(all_tracks2)

10991

In [31]:
tracks22 = json_normalize(all_tracks2)


In [32]:
tracks22.head()


,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls.spotify,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
0,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/5MBUL8d9FAYCCMr...,https://api.spotify.com/v1/albums/5MBUL8d9FAYC...,5MBUL8d9FAYCCMrogOFX68,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",United,2000-03-06,day,10,album,spotify:album:5MBUL8d9FAYCCMrogOFX68,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,266600,False,False,FRS639900098,https://open.spotify.com/track/3AA8xNhDC0Mpqwk...,https://api.spotify.com/v1/tracks/3AA8xNhDC0Mp...,3AA8xNhDC0MpqwkGX3EP5V,False,If I Ever Feel Better,66,https://p.scdn.co/mp3-preview/eb1e1d7972172e23...,True,4,track,spotify:track:3AA8xNhDC0MpqwkGX3EP5V,None
1,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,single,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/6Maw9mmnklMBmaL...,https://api.spotify.com/v1/albums/6Maw9mmnklMB...,6Maw9mmnklMBmaLrl1W6v7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Dance To The Underground,2001,year,4,album,spotify:album:6Maw9mmnklMBmaLrl1W6v7,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,243213,False,False,US2B30405301,https://open.spotify.com/track/2i8lRhmZNKaM1yp...,https://api.spotify.com/v1/tracks/2i8lRhmZNKaM...,2i8lRhmZNKaM1ypHwRybuD,False,Dance To The Underground,29,https://p.scdn.co/mp3-preview/bf9953e29d806a33...,True,1,track,spotify:track:2i8lRhmZNKaM1ypHwRybuD,None
2,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/6IvwFVN9YI9Bokk...,https://api.spotify.com/v1/albums/6IvwFVN9YI9B...,6IvwFVN9YI9Bokka32c80b,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",UltraSex,2002,year,13,album,spotify:album:6IvwFVN9YI9Bokka32c80b,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,234000,False,False,US25T9915124,https://open.spotify.com/track/4flxgPC0426CEeS...,https://api.spotify.com/v1/tracks/4flxgPC0426C...,4flxgPC0426CEeSrzQIic0,False,How We Do,34,https://p.scdn.co/mp3-preview/7b8c0e7b8498eaf4...,True,1,track,spotify:track:4flxgPC0426CEeSrzQIic0,None
3,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/5sgLqGnrMemtvQ5...,https://api.spotify.com/v1/albums/5sgLqGnrMemt...,5sgLqGnrMemtvQ5c4GUhab,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",The Teaches of Peaches,2002-10-08,day,17,album,spotify:album:5sgLqGnrMemtvQ5c4GUhab,[{'extern

In [33]:
# we can even create a new column containing these DataFrames! KEEP song URI
def expand_list_dict2(row):  
    df2 = json_normalize(row['track.artists'])
    df2['song_id'] = row['track.id']
    return df2

tracks22['artists_dfs'] = tracks22.apply(expand_list_dict2, axis=1)
tracks22['artists_dfs'][3]

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/1gkSl4XpHIH...,1gkSl4XpHIHI4I1WQbfXOE,Peaches,artist,spotify:artist:1gkSl4XpHIHI4I1WQbfXOE,https://open.spotify.com/artist/1gkSl4XpHIHI4I...,1XHFob24QklIXtLRopKirJ


In [34]:
# now we create a new dataframe with all these artists
artist_df22 = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri'])
for mini_df in tracks22['artists_dfs']:
    #display(val)
    artist_df22 = pd.concat([artist_df22, mini_df], axis=0)
    
artist_df22

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/1xU878Z1QtB...,1xU878Z1QtBldR7ru9owdU,Phoenix,artist,spotify:artist:1xU878Z1QtBldR7ru9owdU,https://open.spotify.com/artist/1xU878Z1QtBldR...,3AA8xNhDC0MpqwkGX3EP5V
0,https://api.spotify.com/v1/artists/2oWyxHNcxMc...,2oWyxHNcxMcrvNIHMX2apR,Radio 4,artist,spotify:artist:2oWyxHNcxMcrvNIHMX2apR,https://open.spotify.com/artist/2oWyxHNcxMcrvN...,2i8lRhmZNKaM1ypHwRybuD
0,https://api.spotify.com/v1/artists/2UyhhfLOvfL...,2UyhhfLOvfLs7ZhzsAaNC3,Mount Sims,artist,spotify:artist:2UyhhfLOvfLs7ZhzsAaNC3,https://open.spotify.com/artist/2UyhhfLOvfLs7Z...,4flxgPC0426CEeSrzQIic0
0,https://api.spotify.com/v1/artists/1gkSl4XpHIH...,1gkSl4XpHIHI4I1WQbfXOE,Peaches,artist,spotify:artist:1gkSl4XpHIHI4I1WQbfXOE,https://open.spotify.com/artist/1gkSl4XpHIHI4I...,1XHFob24QklIXtLRopKirJ
0,https://api.spotify.com/v1/artists/3GpAL7oEFD3...,3GpAL7oEFD37IJDOOiirqy,Zongamin,artist,spotify:artist:3GpAL7oEFD37IJDOOiirqy,https://open.spotify.com/artist/3GpAL7oEFD37IJ...,4JB847zlgViLq8tJIzRsZy
...,...,...,...,...,...,...,...
0,https://api.spotify.com/v1/artists/4HpHDJqNFP3...,4HpHDJqNFP3XsGRy0SKTq4,SLEEP RADIO,artist,spotify:artist:4HpHDJqNFP3XsGRy0SKTq4,https://open.spotify.com/artist/4HpHDJqNFP3XsG...,1v6vHkDHBjm4OHUzeOKber
0,https://api.spotify.com/v1/artists/2sriRQRt36D...,2sriRQRt36DnfHtD68zZlj,Dorian Concept,artist,spotify:artist:2sriRQRt36DnfHtD68zZlj,https://open.spotify.com/artist/2sriRQRt36DnfH...,5TN8n4LQS8Or2WEnU56pXs
0,https://api.spotify.com/v1/artists/54QMjE4toDf...,54QMjE4toDfiCryzYWCpXX,Metronomy,artist,spotify:artist:54QMjE4toDfiCryzYWCpXX,https://open.spotify.com/artist/54QMjE4toDfiCr...,6MEzr1cmAJi3FXBjXVXMnm
0,https://api.spotify.com/v1/artists/5wFXmYsg3KF...,5wFXmYsg3KFJ8BDsQudJ4f,Manchester Orchestra,artist,spotify:artist:5wFXmYsg3KFJ8BDsQudJ4f,https://open.spotify.com/artist/5wFXmYsg3KFJ8B...,6ovSjbBKatyx9KvWsxkAZo


In [35]:
# now we merge (join) the two dataframes, keeping only the fields we need

df_merged22 = pd.merge(left=tracks22,
                    right=artist_df22,
                    how='inner',
                    left_on='track.id',
                    right_on='song_id')
df_merged22.head()

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls.spotify,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url,artists_dfs,href,id,name,type,uri,external_urls.spotify,song_id
0,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/5MBUL8d9FAYCCMr...,https://api.spotify.com/v1/albums/5MBUL8d9FAYC...,5MBUL8d9FAYCCMrogOFX68,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",United,2000-03-06,day,10,album,spotify:album:5MBUL8d9FAYCCMrogOFX68,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,266600,False,False,FRS639900098,https://open.spotify.com/track/3AA8xNhDC0Mpqwk...,https://api.spotify.com/v1/tracks/3AA8xNhDC0Mp...,3AA8xNhDC0MpqwkGX3EP5V,False,If I Ever Feel Better,66,https://p.scdn.co/mp3-preview/eb1e1d7972172e23...,True,4,track,spotify:track:3AA8xNhDC0MpqwkGX3EP5V,None,...,https://api.spotify.com/v1/artists/1xU878Z1QtB...,1xU878Z1QtBldR7ru9owdU,Phoenix,artist,spotify:artist:1xU878Z1QtBldR7ru9owdU,https://open.spotify.com/artist/1xU878Z1QtBldR...,3AA8xNhDC0MpqwkGX3EP5V
1,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,single,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/6Maw9mmnklMBmaL...,https://api.spotify.com/v1/albums/6Maw9mmnklMB...,6Maw9mmnklMBmaLrl1W6v7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Dance To The Underground,2001,year,4,album,spotify:album:6Maw9mmnklMBmaLrl1W6v7,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,243213,False,False,US2B30405301,https://open.spotify.com/track/2i8lRhmZNKaM1yp...,https://api.spotify.com/v1/tracks/2i8lRhmZNKaM...,2i8lRhmZNKaM1ypHwRybuD,False,Dance To The Underground,29,https://p.scdn.co/mp3-preview/bf9953e29d806a33...,True,1,track,spotify:track:2i8lRhmZNKaM1ypHwRybuD,None,...,https://api.spotify.com/v1/artists/2oWyxHNcxMc...,2oWyxHNcxMcrvNIHMX2apR,Radio 4,artist,spotify:artist:2oWyxHNcxMcrvNIHMX2apR,https://open.spotify.com/artist/2oWyxHNcxMcrvN...,2i8lRhmZNKaM1ypHwRybuD
2,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/6IvwFVN9YI9Bokk...,https://api.spotify.com/v1/albums/6IvwFVN9YI9B...,6IvwFVN9YI9Bokka32c80b,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",UltraSex,2002,year,13,album,spotify:album:6IvwFVN9YI9Bokka32c80b,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,234000,False,False,US25T9915124,https://open.spotify.com/track/4flxgPC0426CEeS...,https://api.spotify.com/v1/tracks/4flxgPC0426C...,4flxgPC0426CEeSrzQIic0,False,How We Do,34,https://p.scdn.co/mp3-preview/7b8c0e7b8498eaf4...,True,1,track,spotify:track:4flxgPC0426CEeSrzQIic0,None,...,https://api.spotify.com/v1/artis

In [36]:
df_final22 = df_merged22[['track.name', 'name', 'song_id']]
df_final22

,track.name,name,song_id
0,If I Ever Feel Better,Phoenix,3AA8xNhDC0MpqwkGX3EP5V
1,Dance To The Underground,Radio 4,2i8lRhmZNKaM1ypHwRybuD
2,How We Do,Mount Sims,4flxgPC0426CEeSrzQIic0
3,Fuck the Pain Away,Peaches,1XHFob24QklIXtLRopKirJ
4,Street Surgery 2,Zongamin,4JB847zlgViLq8tJIzRsZy
...,...,...,...
12676,Hate Is an Attractive Force,SLEEP RADIO,1v6vHkDHBjm4OHUzeOKber
12677,Promises,Dorian Concept,5TN8n4LQS8Or2WEnU56pXs
12678,16 Beat,Metronomy,6MEzr1cmAJi3FXBjXVXMnm
12679,Amplified In The Silence,Manchester Orchestra,6ovSjbBKatyx9KvWsxkAZo


### Audio features

In [37]:
chunks = [(i, i+100) for i in range(0, len(df_final22), 100)]
chunks  
audio_features_list2 = []
for chunk in chunks:
    df_final22['song_id'] = df_final22['song_id'].astype(str)
    id2_list100 = df_final22['song_id'][chunk[0]:chunk[1]] 
    audio_features_list2 = audio_features_list2 + sp.audio_features(id2_list100)
    sleep(randint(1,3000)/1000)
len(audio_features_list2)

C:\Users\laiag\AppData\Local\Temp\ipykernel_20728\3064125078.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final22['song_id'] = df_final22['song_id'].astype(str)


12681

In [38]:
audio_features_df2 = json_normalize(audio_features_list2)


In [39]:
audio_features_df2.drop_duplicates(inplace=True) 


In [40]:
df_w_audio_ft2 = pd.merge(left=df_final22,  #depois voltamos a fazer o merge do audio com a musica
                        right=audio_features_df2,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft2

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,If I Ever Feel Better,Phoenix,3AA8xNhDC0MpqwkGX3EP5V,0.755,0.553,6,-9.449,0,0.1070,0.04920,0.006070,0.0646,0.836,119.700,audio_features,3AA8xNhDC0MpqwkGX3EP5V,spotify:track:3AA8xNhDC0MpqwkGX3EP5V,https://api.spotify.com/v1/tracks/3AA8xNhDC0Mp...,https://api.spotify.com/v1/audio-analysis/3AA8...,266600,4
1,Dance To The Underground,Radio 4,2i8lRhmZNKaM1ypHwRybuD,0.667,0.667,7,-6.267,1,0.0388,0.00013,0.003850,0.0711,0.843,130.321,audio_features,2i8lRhmZNKaM1ypHwRybuD,spotify:track:2i8lRhmZNKaM1ypHwRybuD,https://api.spotify.com/v1/tracks/2i8lRhmZNKaM...,https://api.spotify.com/v1/audio-analysis/2i8l...,243213,4
2,How We Do,Mount Sims,4flxgPC0426CEeSrzQIic0,0.826,0.601,10,-3.966,0,0.0515,0.19500,0.000003,0.0700,0.948,126.971,audio_features,4flxgPC0426CEeSrzQIic0,spotify:track:4flxgPC0426CEeSrzQIic0,https://api.spotify.com/v1/tracks/4flxgPC0426C...,https://api.spotify.com/v1/audio-analysis/4flx...,234000,4
3,Fuck the Pain Away,Peaches,1XHFob24QklIXtLRopKirJ,0.905,0.945,4,-2.630,1,0.1370,0.03540,0.005290,0.0707,0.944,131.597,audio_features,1XHFob24QklIXtLRopKirJ,spotify:track:1XHFob24QklIXtLRopKirJ,https://api.spotify.com/v1/tracks/1XHFob24QklI...,https://api.spotify.com/v1/audio-analysis/1XHF...,250693,4
4,Street Surgery 2,Zongamin,4JB847zlgViLq8tJIzRsZy,0.797,0.536,11,-8.267,0,0.2660,0.03150,0.000029,0.1700,0.704,102.881,audio_features,4JB847zlgViLq8tJIzRsZy,spotify:track:4JB847zlgViLq8tJIzRsZy,https://api.spotify.com/v1/tracks/4JB847zlgViL...,https://api.spotify.com/v1/audio-analysis/4JB8...,224800,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12676,Hate Is an Attractive Force,SLEEP RADIO,1v6vHkDHBjm4OHUzeOKber,0.759,0.121,7,-25.346,0,0.0284,0.41000,0.006100,0.0905,0.523,90.029,audio_features,1v6vHkDHBjm4OHUzeOKber,spotify:track:1v6vHkDHBjm4OHUzeOKber,https://api.spotify.com/v1/tracks/1v6vHkDHBjm4...,https://api.spotify.com/v1/audio-analysis/1v6v...,170667,4
12677,Promises,Dorian Concept,5TN8n4LQS8Or2WEnU56pXs,0.621,0.675,10,-9.347,0,0.0685,0.38500,0.033700,0.3470,0.323,124.056,audio_features,5TN8n4LQS8Or2WEnU56pXs,spotify:track:5TN8n4LQS8Or2WEnU56pXs,https://api.spotify.com/v1/tracks/5TN8n4LQS8Or...,https://api.spotify.com/v1/audio-analysis/5TN8...,237064,1
12678,16 Beat,Metronomy,6MEzr1cmAJi3FXBjXVXMnm,0.781,0.812,7,-11.866,1,0.0468,0.01860,0.586000,0.0716,0.785,115.001,audio_features,6MEzr1cmAJi3FXBjXVXMnm,spotify:track:6MEzr1cmAJi3FXBjXVXMnm,https://api.spotify.com/v1/tracks/6MEzr1cmAJi3...,https://api.spotify.com/v1/audio-analysis/6MEz...,196027,4
12679,Amplified In The Silence,Manchester Orchestra,6ovSjbBKatyx9KvWsxkAZo,0.557,0.117,1,-13.880,1,0.0344,0.95000,0.000269,0.1120,0.122,111.742,audio_features,6ovSjbBKatyx9KvWsxkAZo,spotify:track:6ovSjbBKatyx9KvWsxkAZo,https://api.spotify.com/v1/tracks/6ovSjbBKatyx...,https://api.spotify.com/v1/audio-analysis/6ovS...,223107,4


### Playlist 3: https://open.spotify.com/playlist/5S8SJdl1BDc0ugpkEvFsIL

In [41]:
playlist3 = sp.user_playlist_tracks("spotify", "5S8SJdl1BDc0ugpkEvFsIL")

In [42]:
# Let's look at items and total:
playlist3.keys() 

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [43]:
# let's see how many songs are in this playlist:
playlist3["total"]

10000

In [44]:
# Each page is limited to 100 tracks, we will have to fix it:
len(playlist3["items"])

100

In [45]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # respectful nap
    return tracks

In [46]:
all_tracks3 = get_playlist_tracks("5S8SJdl1BDc0ugpkEvFsIL")
len(all_tracks3)

10000

In [47]:
tracks3 = json_normalize(all_tracks3)

In [48]:
tracks3.head()

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls.spotify,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
0,2017-11-20T02:52:18Z,False,None,https://open.spotify.com/user/twgeb7mzdcv4u8h1...,https://api.spotify.com/v1/users/twgeb7mzdcv4u...,twgeb7mzdcv4u8h191dxrvlpc,user,spotify:user:twgeb7mzdcv4u8h191dxrvlpc,single,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/0HG8fMDhvN2tH5u...,https://api.spotify.com/v1/albums/0HG8fMDhvN2t...,0HG8fMDhvN2tH5uPHFsyZP,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",...Ready For It?,2017-09-03,day,1.0,album,spotify:album:0HG8fMDhvN2tH5uPHFsyZP,[{'external_urls': {'spotify': 'https://open.s...,[],1,208198,False,False,USCJY1750003,https://open.spotify.com/track/7zgqtptZvhf8GEm...,https://api.spotify.com/v1/tracks/7zgqtptZvhf8...,7zgqtptZvhf8GEmdsM2vp2,False,...Ready For It?,0,None,True,1,track,spotify:track:7zgqtptZvhf8GEmdsM2vp2,None
1,2017-11-20T02:42:15Z,False,None,https://open.spotify.com/user/twgeb7mzdcv4u8h1...,https://api.spotify.com/v1/users/twgeb7mzdcv4u...,twgeb7mzdcv4u8h191dxrvlpc,user,spotify:user:twgeb7mzdcv4u8h191dxrvlpc,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/4w5Jvreahp3yvLq...,https://api.spotify.com/v1/albums/4w5Jvreahp3y...,4w5Jvreahp3yvLqc4vCr9I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Life Changes,2017-09-08,day,14.0,album,spotify:album:4w5Jvreahp3yvLqc4vCr9I,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,190226,False,False,USLXJ1707376,https://open.spotify.com/track/4Vxu50qVrQcycjR...,https://api.spotify.com/v1/tracks/4Vxu50qVrQcy...,4Vxu50qVrQcycjRyJQaZLC,False,Life Changes,62,None,True,8,track,spotify:track:4Vxu50qVrQcycjRyJQaZLC,None
2,2017-11-20T02:36:40Z,False,None,https://open.spotify.com/user/twgeb7mzdcv4u8h1...,https://api.spotify.com/v1/users/twgeb7mzdcv4u...,twgeb7mzdcv4u8h191dxrvlpc,user,spotify:user:twgeb7mzdcv4u8h191dxrvlpc,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/4PgleR09JVnm3zY...,https://api.spotify.com/v1/albums/4PgleR09JVnm...,4PgleR09JVnm3zY1fW3XBA,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",24K Magic,2016-11-17,day,9.0,album,spotify:album:4PgleR09JVnm3zY1fW3XBA,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,225983,False,False,USAT21602944,https://open.spotify.com/track/6b8Be6ljOzmkOmF...,https://api.spotify.com/v1/tracks/6b8Be6ljOzmk...,6b8Be6ljOzmkOmFslEb23P,False,24K Magic,82,https://p.scdn.co/mp3-preview/3a76820d510fa5f8...,True,1,track,spotify:track:6b8Be6ljOzmkOmFslEb23P,None
3,2017-12-06T01:26:36Z,False,None,https://open.spotify.com/user/twgeb7mzdcv4u8h1...,https://api.spotify.com/v1/users/twgeb7mzdcv4u...,twgeb7mzdcv4u8h191dxrvlpc,user,spotify:user:twgeb7mzdcv4u8h191dxrvlpc,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/3T4tUhGYeRNVUGe...,https://api.spotify.com/v1/albums/3T4tUhGYeRNV...,3T4tUhGYeRNVUGevb0wThu,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",÷ (Deluxe),2017-03-03,day,16.0,album,spotify:album:3T4tUhGYeRNVUGevb0wThu,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT

In [49]:
# we can even create a new column containing these DataFrames! KEEP song URI
def expand_list_dict(row):
    df = json_normalize(row['track.artists'])
    df['song_id'] = row['track.id']
    return df

tracks3['artists_dfs'] = tracks3.apply(expand_list_dict, axis=1)
tracks3['artists_dfs'][3]

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/6eUKZXaKkcv...,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran,artist,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,https://open.spotify.com/artist/6eUKZXaKkcviH0...,0afhq8XCExXpqazXczTSve


In [50]:
# now we create a new dataframe with all these artists
artist_df3 = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri'])
for mini_df in tracks3['artists_dfs']:
    #display(val)
    artist_df3 = pd.concat([artist_df3, mini_df], axis=0)
    
artist_df3

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/06HL4z0CvFA...,06HL4z0CvFAxyc27GXpf02,Taylor Swift,artist,spotify:artist:06HL4z0CvFAxyc27GXpf02,https://open.spotify.com/artist/06HL4z0CvFAxyc...,7zgqtptZvhf8GEmdsM2vp2
0,https://api.spotify.com/v1/artists/6x2LnllRG5u...,6x2LnllRG5uGarZMsD4iO8,Thomas Rhett,artist,spotify:artist:6x2LnllRG5uGarZMsD4iO8,https://open.spotify.com/artist/6x2LnllRG5uGar...,4Vxu50qVrQcycjRyJQaZLC
0,https://api.spotify.com/v1/artists/0du5cEVh5yT...,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,artist,spotify:artist:0du5cEVh5yTK9QJze8zA0C,https://open.spotify.com/artist/0du5cEVh5yTK9Q...,6b8Be6ljOzmkOmFslEb23P
0,https://api.spotify.com/v1/artists/6eUKZXaKkcv...,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran,artist,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,https://open.spotify.com/artist/6eUKZXaKkcviH0...,0afhq8XCExXpqazXczTSve
0,https://api.spotify.com/v1/artists/6eUKZXaKkcv...,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran,artist,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,https://open.spotify.com/artist/6eUKZXaKkcviH0...,1HNkqx9Ahdgi1Ixy2xkKkL
...,...,...,...,...,...,...,...
0,https://api.spotify.com/v1/artists/3JsMj0DEzyW...,3JsMj0DEzyWc0VDlHuy9Bx,Supertramp,artist,spotify:artist:3JsMj0DEzyWc0VDlHuy9Bx,https://open.spotify.com/artist/3JsMj0DEzyWc0V...,5pSSEkT0963muzzIjsVkrs
0,https://api.spotify.com/v1/artists/7AC976RDJzL...,7AC976RDJzL2asmZuz7qil,Yes,artist,spotify:artist:7AC976RDJzL2asmZuz7qil,https://open.spotify.com/artist/7AC976RDJzL2as...,7gC6Rbllqf1yXNC02e5jz2
0,https://api.spotify.com/v1/artists/4STHEaNw4mP...,4STHEaNw4mPZ2tzheohgXB,Paul McCartney,artist,spotify:artist:4STHEaNw4mPZ2tzheohgXB,https://open.spotify.com/artist/4STHEaNw4mPZ2t...,6Ff77WXC58MkhLE5A1qgY1
1,https://api.spotify.com/v1/artists/3sFhA6G1N0g...,3sFhA6G1N0gG1pszb6kk1m,Wings,artist,spotify:artist:3sFhA6G1N0gG1pszb6kk1m,https://open.spotify.com/artist/3sFhA6G1N0gG1p...,6Ff77WXC58MkhLE5A1qgY1


In [51]:
# now we merge (join) the two dataframes, keeping only the fields we need
df_merged3 = pd.merge(left=tracks3,
                    right=artist_df3,
                    how='inner',
                    left_on='track.id',
                    right_on='song_id')
df_merged3.head()

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls.spotify,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url,artists_dfs,href,id,name,type,uri,external_urls.spotify,song_id
0,2017-11-20T02:52:18Z,False,None,https://open.spotify.com/user/twgeb7mzdcv4u8h1...,https://api.spotify.com/v1/users/twgeb7mzdcv4u...,twgeb7mzdcv4u8h191dxrvlpc,user,spotify:user:twgeb7mzdcv4u8h191dxrvlpc,single,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/0HG8fMDhvN2tH5u...,https://api.spotify.com/v1/albums/0HG8fMDhvN2t...,0HG8fMDhvN2tH5uPHFsyZP,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",...Ready For It?,2017-09-03,day,1.0,album,spotify:album:0HG8fMDhvN2tH5uPHFsyZP,[{'external_urls': {'spotify': 'https://open.s...,[],1,208198,False,False,USCJY1750003,https://open.spotify.com/track/7zgqtptZvhf8GEm...,https://api.spotify.com/v1/tracks/7zgqtptZvhf8...,7zgqtptZvhf8GEmdsM2vp2,False,...Ready For It?,0,None,True,1,track,spotify:track:7zgqtptZvhf8GEmdsM2vp2,None,...,https://api.spotify.com/v1/artists/06HL4z0CvFA...,06HL4z0CvFAxyc27GXpf02,Taylor Swift,artist,spotify:artist:06HL4z0CvFAxyc27GXpf02,https://open.spotify.com/artist/06HL4z0CvFAxyc...,7zgqtptZvhf8GEmdsM2vp2
1,2017-11-20T02:42:15Z,False,None,https://open.spotify.com/user/twgeb7mzdcv4u8h1...,https://api.spotify.com/v1/users/twgeb7mzdcv4u...,twgeb7mzdcv4u8h191dxrvlpc,user,spotify:user:twgeb7mzdcv4u8h191dxrvlpc,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/4w5Jvreahp3yvLq...,https://api.spotify.com/v1/albums/4w5Jvreahp3y...,4w5Jvreahp3yvLqc4vCr9I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Life Changes,2017-09-08,day,14.0,album,spotify:album:4w5Jvreahp3yvLqc4vCr9I,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,190226,False,False,USLXJ1707376,https://open.spotify.com/track/4Vxu50qVrQcycjR...,https://api.spotify.com/v1/tracks/4Vxu50qVrQcy...,4Vxu50qVrQcycjRyJQaZLC,False,Life Changes,62,None,True,8,track,spotify:track:4Vxu50qVrQcycjRyJQaZLC,None,...,https://api.spotify.com/v1/artists/6x2LnllRG5u...,6x2LnllRG5uGarZMsD4iO8,Thomas Rhett,artist,spotify:artist:6x2LnllRG5uGarZMsD4iO8,https://open.spotify.com/artist/6x2LnllRG5uGar...,4Vxu50qVrQcycjRyJQaZLC
2,2017-11-20T02:36:40Z,False,None,https://open.spotify.com/user/twgeb7mzdcv4u8h1...,https://api.spotify.com/v1/users/twgeb7mzdcv4u...,twgeb7mzdcv4u8h191dxrvlpc,user,spotify:user:twgeb7mzdcv4u8h191dxrvlpc,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/4PgleR09JVnm3zY...,https://api.spotify.com/v1/albums/4PgleR09JVnm...,4PgleR09JVnm3zY1fW3XBA,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",24K Magic,2016-11-17,day,9.0,album,spotify:album:4PgleR09JVnm3zY1fW3XBA,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,225983,False,False,USAT21602944,https://open.spotify.com/track/6b8Be6ljOzmkOmF...,https://api.spotify.com/v1/tracks/6b8Be6ljOzmk...,6b8Be6ljOzmkOmFslEb23P,False,24K Magic,82,https://p.scdn.co/mp3-preview/3a76820d510fa5f8...,True,1,track,spotify:track:6b8Be6ljOzmkOmFslEb23P,None,...,https://api.spotify.com/v1/artists/0du5cEVh5yT...,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,artist,spotify:artist:0du5cEVh5yTK9QJze8zA0C,https://op

In [52]:
df_final3 = df_merged3[['track.name', 'name', 'song_id']]
df_final3

,track.name,name,song_id
0,...Ready For It?,Taylor Swift,7zgqtptZvhf8GEmdsM2vp2
1,Life Changes,Thomas Rhett,4Vxu50qVrQcycjRyJQaZLC
2,24K Magic,Bruno Mars,6b8Be6ljOzmkOmFslEb23P
3,Galway Girl,Ed Sheeran,0afhq8XCExXpqazXczTSve
4,Photograph,Ed Sheeran,1HNkqx9Ahdgi1Ixy2xkKkL
...,...,...,...
12609,Fool's Overture,Supertramp,5pSSEkT0963muzzIjsVkrs
12610,Heart of the Sunrise - 2003 Remaster,Yes,7gC6Rbllqf1yXNC02e5jz2
12611,Venus And Mars / Rock Show / Jet - Live / Rema...,Paul McCartney,6Ff77WXC58MkhLE5A1qgY1
12612,Venus And Mars / Rock Show / Jet - Live / Rema...,Wings,6Ff77WXC58MkhLE5A1qgY1


### Audio features

In [53]:
chunks = [(i, i+100) for i in range(0, len(df_final3), 100)]
chunks
audio_features_list3 = []
for chunk in chunks:
    df_final3['song_id'] = df_final3['song_id'].astype(str)
    id2_list100 = df_final3['song_id'][chunk[0]:chunk[1]]
    audio_features_list3 = audio_features_list3 + sp.audio_features(id2_list100)
    sleep(randint(1,3000)/1000)
len(audio_features_list3)

C:\Users\laiag\AppData\Local\Temp\ipykernel_20728\3971827008.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final3['song_id'] = df_final3['song_id'].astype(str)


12614

In [54]:
audio_features_df3 = json_normalize(audio_features_list3)

In [55]:
audio_features_df3.drop_duplicates(inplace=True) 

In [56]:
df_w_audio_ft3 = pd.merge(left=df_final3,
                        right=audio_features_df3,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft3

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,...Ready For It?,Taylor Swift,7zgqtptZvhf8GEmdsM2vp2,0.615,0.779,2.0,-6.454,1.0,0.1350,0.06650,0.000000,0.1550,0.453,160.000,audio_features,7zgqtptZvhf8GEmdsM2vp2,spotify:track:7zgqtptZvhf8GEmdsM2vp2,https://api.spotify.com/v1/tracks/7zgqtptZvhf8...,https://api.spotify.com/v1/audio-analysis/7zgq...,208198.0,4.0
1,Life Changes,Thomas Rhett,4Vxu50qVrQcycjRyJQaZLC,0.687,0.845,7.0,-4.370,1.0,0.0576,0.10000,0.000000,0.0452,0.809,87.972,audio_features,4Vxu50qVrQcycjRyJQaZLC,spotify:track:4Vxu50qVrQcycjRyJQaZLC,https://api.spotify.com/v1/tracks/4Vxu50qVrQcy...,https://api.spotify.com/v1/audio-analysis/4Vxu...,190227.0,4.0
2,24K Magic,Bruno Mars,6b8Be6ljOzmkOmFslEb23P,0.818,0.803,1.0,-4.282,1.0,0.0797,0.03400,0.000000,0.1530,0.632,106.970,audio_features,6b8Be6ljOzmkOmFslEb23P,spotify:track:6b8Be6ljOzmkOmFslEb23P,https://api.spotify.com/v1/tracks/6b8Be6ljOzmk...,https://api.spotify.com/v1/audio-analysis/6b8B...,225983.0,4.0
3,Galway Girl,Ed Sheeran,0afhq8XCExXpqazXczTSve,0.624,0.876,9.0,-3.374,1.0,0.1000,0.07350,0.000000,0.3270,0.781,99.943,audio_features,0afhq8XCExXpqazXczTSve,spotify:track:0afhq8XCExXpqazXczTSve,https://api.spotify.com/v1/tracks/0afhq8XCExXp...,https://api.spotify.com/v1/audio-analysis/0afh...,170827.0,4.0
4,Photograph,Ed Sheeran,1HNkqx9Ahdgi1Ixy2xkKkL,0.614,0.379,4.0,-10.480,1.0,0.0476,0.60700,0.000464,0.0986,0.201,107.989,audio_features,1HNkqx9Ahdgi1Ixy2xkKkL,spotify:track:1HNkqx9Ahdgi1Ixy2xkKkL,https://api.spotify.com/v1/tracks/1HNkqx9Ahdgi...,https://api.spotify.com/v1/audio-analysis/1HNk...,258987.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12180,Fool's Overture,Supertramp,5pSSEkT0963muzzIjsVkrs,0.406,0.306,3.0,-10.482,1.0,0.0372,0.31300,0.007900,0.0727,0.073,135.272,audio_features,5pSSEkT0963muzzIjsVkrs,spotify:track:5pSSEkT0963muzzIjsVkrs,https://api.spotify.com/v1/tracks/5pSSEkT0963m...,https://api.spotify.com/v1/audio-analysis/5pSS...,652560.0,4.0
12181,Heart of the Sunrise - 2003 Remaster,Yes,7gC6Rbllqf1yXNC02e5jz2,0.362,0.507,1.0,-11.229,1.0,0.0394,0.01740,0.216000,0.1130,0.456,146.641,audio_features,7gC6Rbllqf1yXNC02e5jz2,spotify:track:7gC6Rbllqf1yXNC02e5jz2,https://api.spotify.com/v1/tracks/7gC6Rbllqf1y...,https://api.spotify.com/v1/audio-analysis/7gC6...,634440.0,3.0
12182,Venus And Mars / Rock Show / Jet - Live / Rema...,Paul McCartney,6Ff77WXC58MkhLE5A1qgY1,0.331,0.733,2.0,-8.671,1.0,0.0468,0.08870,0.001740,0.9470,0.380,128.512,audio_features,6Ff77WXC58MkhLE5A1qgY1,spotify:track:6Ff77WXC58MkhLE5A1qgY1,https://api.spotify.com/v1/tracks/6Ff77WXC58Mk...,https://api.spotify.com/v1/audio-analysis/6Ff7...,620747.0,4.0
12183,Venus And Mars / Rock Show / Jet - Live / Rema...,Wings,6Ff77WXC58MkhLE5A1qgY1,0.331,0.733,2.0,-8.671,1.0,0.0468,0.08870,0.001740,0.9470,0.380,128.512,audio_features,6Ff77WXC58MkhLE5A1qgY1,spotify:track:6Ff77WXC58MkhLE5A1qgY1,https://api.spotify.com/v1/tracks/6Ff77WXC58Mk...,https://api.spotify.com/v1/audio-analysis/6Ff7...,620747.0,4.0


### Final Playlist

In [58]:
df_w_audio_ftotal = pd.concat([df_w_audio_ft1, df_w_audio_ft2, df_w_audio_ft3], axis=0, ignore_index=True)

df_w_audio_ftotal

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,2K,Nosaj Thing,33xMbeHzmWd6Od0BmLZEUs,0.310,0.445,7.0,-13.355,0.0,0.0863,0.09400,0.06780,0.1130,0.122,95.360,audio_features,33xMbeHzmWd6Od0BmLZEUs,spotify:track:33xMbeHzmWd6Od0BmLZEUs,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,https://api.spotify.com/v1/audio-analysis/33xM...,152560.0,3.0
1,4 Billion Souls,The Doors,3UnyplmZaq547hwsfOR5yy,0.419,0.565,5.0,-11.565,1.0,0.0347,0.13700,0.33700,0.1280,0.648,151.277,audio_features,3UnyplmZaq547hwsfOR5yy,spotify:track:3UnyplmZaq547hwsfOR5yy,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,https://api.spotify.com/v1/audio-analysis/3Uny...,197707.0,4.0
2,4 Minute Warning,Radiohead,1w8QCSDH4QobcQeT4uMKLm,0.354,0.302,9.0,-13.078,1.0,0.0326,0.59000,0.07090,0.1110,0.223,123.753,audio_features,1w8QCSDH4QobcQeT4uMKLm,spotify:track:1w8QCSDH4QobcQeT4uMKLm,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,https://api.spotify.com/v1/audio-analysis/1w8Q...,244285.0,4.0
3,7 Element,Vitas,7J9mBHG4J2eIfDAv5BehKA,0.727,0.785,5.0,-6.707,0.0,0.0603,0.32500,0.12600,0.3100,0.960,129.649,audio_features,7J9mBHG4J2eIfDAv5BehKA,spotify:track:7J9mBHG4J2eIfDAv5BehKA,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,https://api.spotify.com/v1/audio-analysis/7J9m...,249940.0,4.0
4,#9 Dream,R.E.M.,1VZedwJj1gyi88WFRhfThb,0.571,0.724,0.0,-5.967,1.0,0.0260,0.02310,0.00311,0.0919,0.385,116.755,audio_features,1VZedwJj1gyi88WFRhfThb,spotify:track:1VZedwJj1gyi88WFRhfThb,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,https://api.spotify.com/v1/audio-analysis/1VZe...,278320.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36733,Fool's Overture,Supertramp,5pSSEkT0963muzzIjsVkrs,0.406,0.306,3.0,-10.482,1.0,0.0372,0.31300,0.00790,0.0727,0.073,135.272,audio_features,5pSSEkT0963muzzIjsVkrs,spotify:track:5pSSEkT0963muzzIjsVkrs,https://api.spotify.com/v1/tracks/5pSSEkT0963m...,https://api.spotify.com/v1/audio-analysis/5pSS...,652560.0,4.0
36734,Heart of the Sunrise - 2003 Remaster,Yes,7gC6Rbllqf1yXNC02e5jz2,0.362,0.507,1.0,-11.229,1.0,0.0394,0.01740,0.21600,0.1130,0.456,146.641,audio_features,7gC6Rbllqf1yXNC02e5jz2,spotify:track:7gC6Rbllqf1yXNC02e5jz2,https://api.spotify.com/v1/tracks/7gC6Rbllqf1y...,https://api.spotify.com/v1/audio-analysis/7gC6...,634440.0,3.0
36735,Venus And Mars / Rock Show / Jet - Live / Rema...,Paul McCartney,6Ff77WXC58MkhLE5A1qgY1,0.331,0.733,2.0,-8.671,1.0,0.0468,0.08870,0.00174,0.9470,0.380,128.512,audio_features,6Ff77WXC58MkhLE5A1qgY1,spotify:track:6Ff77WXC58MkhLE5A1qgY1,https://api.spotify.com/v1/tracks/6Ff77WXC58Mk...,https://api.spotify.com/v1/audio-analysis/6Ff7...,620747.0,4.0
36736,Venus And Mars / Rock Show / Jet - Live / Rema...,Wings,6Ff77WXC58MkhLE5A1qgY1,0.331,0.733,2.0,-8.671,1.0,0.0468,0.08870,0.00174,0.9470,0.380,128.512,audio_features,6Ff77WXC58MkhLE5A1qgY1,spotify:track:6Ff77WXC58MkhLE5A1qgY1,https://api.spotify.com/v1/tracks/6Ff77WXC58Mk...,https://api.spotify.com/v1/audio-analysis/6Ff7...,620747.0,4.0


In [59]:
df_w_audio_ftotal.drop_duplicates(inplace=True)


In [60]:
df_w_audio_ftotal

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,2K,Nosaj Thing,33xMbeHzmWd6Od0BmLZEUs,0.310,0.445,7.0,-13.355,0.0,0.0863,0.09400,0.06780,0.1130,0.122,95.360,audio_features,33xMbeHzmWd6Od0BmLZEUs,spotify:track:33xMbeHzmWd6Od0BmLZEUs,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,https://api.spotify.com/v1/audio-analysis/33xM...,152560.0,3.0
1,4 Billion Souls,The Doors,3UnyplmZaq547hwsfOR5yy,0.419,0.565,5.0,-11.565,1.0,0.0347,0.13700,0.33700,0.1280,0.648,151.277,audio_features,3UnyplmZaq547hwsfOR5yy,spotify:track:3UnyplmZaq547hwsfOR5yy,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,https://api.spotify.com/v1/audio-analysis/3Uny...,197707.0,4.0
2,4 Minute Warning,Radiohead,1w8QCSDH4QobcQeT4uMKLm,0.354,0.302,9.0,-13.078,1.0,0.0326,0.59000,0.07090,0.1110,0.223,123.753,audio_features,1w8QCSDH4QobcQeT4uMKLm,spotify:track:1w8QCSDH4QobcQeT4uMKLm,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,https://api.spotify.com/v1/audio-analysis/1w8Q...,244285.0,4.0
3,7 Element,Vitas,7J9mBHG4J2eIfDAv5BehKA,0.727,0.785,5.0,-6.707,0.0,0.0603,0.32500,0.12600,0.3100,0.960,129.649,audio_features,7J9mBHG4J2eIfDAv5BehKA,spotify:track:7J9mBHG4J2eIfDAv5BehKA,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,https://api.spotify.com/v1/audio-analysis/7J9m...,249940.0,4.0
4,#9 Dream,R.E.M.,1VZedwJj1gyi88WFRhfThb,0.571,0.724,0.0,-5.967,1.0,0.0260,0.02310,0.00311,0.0919,0.385,116.755,audio_features,1VZedwJj1gyi88WFRhfThb,spotify:track:1VZedwJj1gyi88WFRhfThb,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,https://api.spotify.com/v1/audio-analysis/1VZe...,278320.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36733,Fool's Overture,Supertramp,5pSSEkT0963muzzIjsVkrs,0.406,0.306,3.0,-10.482,1.0,0.0372,0.31300,0.00790,0.0727,0.073,135.272,audio_features,5pSSEkT0963muzzIjsVkrs,spotify:track:5pSSEkT0963muzzIjsVkrs,https://api.spotify.com/v1/tracks/5pSSEkT0963m...,https://api.spotify.com/v1/audio-analysis/5pSS...,652560.0,4.0
36734,Heart of the Sunrise - 2003 Remaster,Yes,7gC6Rbllqf1yXNC02e5jz2,0.362,0.507,1.0,-11.229,1.0,0.0394,0.01740,0.21600,0.1130,0.456,146.641,audio_features,7gC6Rbllqf1yXNC02e5jz2,spotify:track:7gC6Rbllqf1yXNC02e5jz2,https://api.spotify.com/v1/tracks/7gC6Rbllqf1y...,https://api.spotify.com/v1/audio-analysis/7gC6...,634440.0,3.0
36735,Venus And Mars / Rock Show / Jet - Live / Rema...,Paul McCartney,6Ff77WXC58MkhLE5A1qgY1,0.331,0.733,2.0,-8.671,1.0,0.0468,0.08870,0.00174,0.9470,0.380,128.512,audio_features,6Ff77WXC58MkhLE5A1qgY1,spotify:track:6Ff77WXC58MkhLE5A1qgY1,https://api.spotify.com/v1/tracks/6Ff77WXC58Mk...,https://api.spotify.com/v1/audio-analysis/6Ff7...,620747.0,4.0
36736,Venus And Mars / Rock Show / Jet - Live / Rema...,Wings,6Ff77WXC58MkhLE5A1qgY1,0.331,0.733,2.0,-8.671,1.0,0.0468,0.08870,0.00174,0.9470,0.380,128.512,audio_features,6Ff77WXC58MkhLE5A1qgY1,spotify:track:6Ff77WXC58MkhLE5A1qgY1,https://api.spotify.com/v1/tracks/6Ff77WXC58Mk...,https://api.spotify.com/v1/audio-analysis/6Ff7...,620747.0,4.0


In [66]:
df_w_audio_ftotal.reset_index(drop=True, inplace=True)
df_w_audio_ftotal

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,2K,Nosaj Thing,33xMbeHzmWd6Od0BmLZEUs,0.310,0.445,7.0,-13.355,0.0,0.0863,0.09400,0.06780,0.1130,0.122,95.360,audio_features,33xMbeHzmWd6Od0BmLZEUs,spotify:track:33xMbeHzmWd6Od0BmLZEUs,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,https://api.spotify.com/v1/audio-analysis/33xM...,152560.0,3.0
1,4 Billion Souls,The Doors,3UnyplmZaq547hwsfOR5yy,0.419,0.565,5.0,-11.565,1.0,0.0347,0.13700,0.33700,0.1280,0.648,151.277,audio_features,3UnyplmZaq547hwsfOR5yy,spotify:track:3UnyplmZaq547hwsfOR5yy,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,https://api.spotify.com/v1/audio-analysis/3Uny...,197707.0,4.0
2,4 Minute Warning,Radiohead,1w8QCSDH4QobcQeT4uMKLm,0.354,0.302,9.0,-13.078,1.0,0.0326,0.59000,0.07090,0.1110,0.223,123.753,audio_features,1w8QCSDH4QobcQeT4uMKLm,spotify:track:1w8QCSDH4QobcQeT4uMKLm,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,https://api.spotify.com/v1/audio-analysis/1w8Q...,244285.0,4.0
3,7 Element,Vitas,7J9mBHG4J2eIfDAv5BehKA,0.727,0.785,5.0,-6.707,0.0,0.0603,0.32500,0.12600,0.3100,0.960,129.649,audio_features,7J9mBHG4J2eIfDAv5BehKA,spotify:track:7J9mBHG4J2eIfDAv5BehKA,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,https://api.spotify.com/v1/audio-analysis/7J9m...,249940.0,4.0
4,#9 Dream,R.E.M.,1VZedwJj1gyi88WFRhfThb,0.571,0.724,0.0,-5.967,1.0,0.0260,0.02310,0.00311,0.0919,0.385,116.755,audio_features,1VZedwJj1gyi88WFRhfThb,spotify:track:1VZedwJj1gyi88WFRhfThb,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,https://api.spotify.com/v1/audio-analysis/1VZe...,278320.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34378,Fool's Overture,Supertramp,5pSSEkT0963muzzIjsVkrs,0.406,0.306,3.0,-10.482,1.0,0.0372,0.31300,0.00790,0.0727,0.073,135.272,audio_features,5pSSEkT0963muzzIjsVkrs,spotify:track:5pSSEkT0963muzzIjsVkrs,https://api.spotify.com/v1/tracks/5pSSEkT0963m...,https://api.spotify.com/v1/audio-analysis/5pSS...,652560.0,4.0
34379,Heart of the Sunrise - 2003 Remaster,Yes,7gC6Rbllqf1yXNC02e5jz2,0.362,0.507,1.0,-11.229,1.0,0.0394,0.01740,0.21600,0.1130,0.456,146.641,audio_features,7gC6Rbllqf1yXNC02e5jz2,spotify:track:7gC6Rbllqf1yXNC02e5jz2,https://api.spotify.com/v1/tracks/7gC6Rbllqf1y...,https://api.spotify.com/v1/audio-analysis/7gC6...,634440.0,3.0
34380,Venus And Mars / Rock Show / Jet - Live / Rema...,Paul McCartney,6Ff77WXC58MkhLE5A1qgY1,0.331,0.733,2.0,-8.671,1.0,0.0468,0.08870,0.00174,0.9470,0.380,128.512,audio_features,6Ff77WXC58MkhLE5A1qgY1,spotify:track:6Ff77WXC58MkhLE5A1qgY1,https://api.spotify.com/v1/tracks/6Ff77WXC58Mk...,https://api.spotify.com/v1/audio-analysis/6Ff7...,620747.0,4.0
34381,Venus And Mars / Rock Show / Jet - Live / Rema...,Wings,6Ff77WXC58MkhLE5A1qgY1,0.331,0.733,2.0,-8.671,1.0,0.0468,0.08870,0.00174,0.9470,0.380,128.512,audio_features,6Ff77WXC58MkhLE5A1qgY1,spotify:track:6Ff77WXC58MkhLE5A1qgY1,https://api.spotify.com/v1/tracks/6Ff77WXC58Mk...,https://api.spotify.com/v1/audio-analysis/6Ff7...,620747.0,4.0


In [68]:
df_w_audio_ftotal.to_csv('df_playlist.csv', index=False)